In [1]:
import requests
import pandas as pd
import time 
from datetime import datetime
from league_summary_helper_functions import *

#Pip install sleeper-api-wrapper if you don't have it already
from sleeper_wrapper import League
from sleeper_wrapper import Players
from sleeper_wrapper import Stats

#Load Credentials
import json
# Load the configuration from config.json
with open('config.json') as config_file:
    config_data = json.load(config_file)

## I. Acquire League Data via. Sleeper

In [2]:
#Identify the week
week = get_NFL_week()
#Load the League data 
league = League(config_data['sleeper']['league_id'])
#Load the rosters
rosters = league.get_rosters()
#Load the players
players = Players()

In [3]:
owners_df = owners(league)
player_dict = player_info(players)
#Import rosters
rosters_df = pd.DataFrame(rosters)
rosters_df = rosters_df[['roster_id', 'owner_id', 'starters','players']]
rosters_df = owners_df[['owner_id', 'owner']].merge(rosters_df, on='owner_id')

In [4]:
#Set up Standings
standings = league.get_standings(league.get_rosters(), league.get_users())
standings_df = pd.DataFrame(standings)
#Add column names
standings_df.columns = ['team_name', 'wins', 'losses', 'points_scored']
standings_df = owners_df[['owner', 'team_name']].merge(standings_df, on='team_name')
#Add an empty column called 'lowest_scoring_team'
standings_df['lowest_scoring_team'] = 0

In [5]:
#Get the latest weekly matchup
matchup_df = weekly_matchup(week, rosters_df, player_dict, league)
updated_standings = iterate_weeks(week, standings_df, weekly_matchup, rosters_df, player_dict, league)
updated_standings

,owner,team_name,wins,losses,points_scored,lowest_scoring_team,median_weekly_score
11,Andrew,andydozntcare,1,0,148,0,148.28
9,Kai,Not Sketch,1,0,121,0,121.28
4,Kyle,GOAT Dynasty,1,0,120,0,120.64
7,David,Korean Jesus,1,0,110,0,110.38
10,Dan,Prototypical NFL Corner,1,0,110,0,110.68
3,Henry,haamsuplo21,1,0,108,0,108.40
8,Tom,Olave Garden,0,1,91,0,91.48
0,Peter,Trust the Process,0,1,76,0,76.44
1,Richard,RAVENS KICKERS & D & O,0,1,69,1,69.44
2,Johnny,ChubbHub,0,1,119,0,119.38


In [27]:
#Run the Bounty Functions
HT_Owner, HT_Score = highest_scoring_team_sleeper(matchup_df)
HP_Owner, HP_Player, HP_Score = highest_scoring_player_sleeper(matchup_df)

## II. Feed Data into GPT

In [7]:
import openai
openai.api_key = config_data['openai']['api_key']

In [9]:
#Feed in formatted data of the weekly matchups and standings and ask GPT to draft a weekly summary email to the league

from tabulate import tabulate
matchup_tabulate = tabulate(matchup_df, headers='keys', tablefmt='plain', showindex=False)
standings_tabulate = tabulate(updated_standings, headers='keys', tablefmt='plain', showindex=False)

In [10]:
input_prompt = f"Week:{week}, \nMatchup Table: \n\n{matchup_tabulate}\n\nStandings Table: \n\n{standings_tabulate}\n\nSummary:"

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "You are an AI Fantasy Football commissioner tasked with writing a weekly summary to your league mates recapping the latest week of our Dynasty league which is in it's first year.\n\nI will provide you a table of the weekly matchups, which includes the owners, their matchup_ids (owners with the same matchup IDs are opponents for the week), their players and what they scored, and a standings table with everyone's records. \nUsing this information, I would like for you to write an email recapping the league in the style of Bill Simmons. Comment on individual match-ups, teams and players that scored well, and poke fun at the worst performing teams. Make the tone funny, light-hearted and slightly sarcastic. "
    },
    {
      "role": "user",
      "content": input_prompt
    }
  ],
  temperature=1,
  max_tokens=2000,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response['choices'][0]['message']['content'])

Subject: Week 1 Recap: Ups, Downs, and the Agony of Defeat

Hey there, Dynasty leaguers!

We are officially underway in our first year of Dynasty football, and Week 1 did not disappoint. From unexpected performances to heartbreaking losses, there was plenty of drama to go around. Let's dive into the action, shall we?

First off, let's give a shoutout to our undefeated owners who are feeling on top of the world right now. Andrew's team, aptly named "andydozntcare," came out swinging with a league-leading 148 points. With Tyreek Hill putting up a ridiculous 39 points and Aaron Jones doing his thing with 25.7, Andrew's squad proved they mean business. Kai, owner of "Not Sketch," followed closely behind with 121 points, thanks in large part to Christian McCaffrey's monster game (24.4 points) and a solid performance from his entire team. And let's not forget about Kyle and his "GOAT Dynasty" team, which put up an impressive 120 points. Looks like the GOAT might just live up to the name afte

## III. Draft Summary Email via Gmail

In [33]:
owners_dict = {
    'Peter': 'peteryushunli@gmail.com',
    'Richard': 'Richardliang2015@gmail.com',
    'Johnny': 'choi.john91@gmail.com',
    'Henry': 'henryzhang21@gmail.com',
    'Kyle': 'komearadoc@gmail.com',
    'Scott': 'lin.scott.bhs@gmail.com',
    'JD': 'johndong2015@u.northwestern.edu',
    'David': 'dchoi26273@gmail.com',
    'Tom': 'tzhangbhs@gmail.com',
    'Kai': 'Kaishen2@gmail.com',
    'Dan': 'jiangd93@gmail.com',
    'Andrew': 'Andydozntcare@gmail.com'
}

In [35]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Load the user name and password from yaml file
user, password = config_data['gmail']['GMAIL_USER'], config_data['gmail']['GMAIL_PW']

matchup = matchup_df.to_html(index=False)
standings = updated_standings.to_html(index=False)

# Initialize the server
s = smtplib.SMTP('smtp.gmail.com', 587)
s.starttls()
s.login(user, password)

# Define message
subject = f"Fagudas Dynasty FF Week {week} Recap"
summary = response['choices'][0]['message']['content']
summary_html = summary.replace('\n', '<br>')
body = f"""\
<html>
  <head></head>
  <body>
    <h1>{subject}</h1>
    <br>{summary_html}</br>
    <br>
    <p>Highest Scoring Team: {HT_Owner}: {HT_Score} points</p>
    <p>Highest Scoring Player: {HP_Owner} - {HP_Player}: {HP_Score} Points</p>
    <br>
    {standings}
  </body>
</html>
"""

recipients = list(owners_dict.values())
sender = "peteryushunli@gmail.com"

msg = MIMEMultipart("alternative")
msg["From"] = sender
msg["To"] = ", ".join(recipients)
msg["Subject"] = subject

# Attach the HTML body
msg.attach(MIMEText(body, 'html'))

# Send email
s.sendmail(sender, recipients, msg.as_string())

# Terminate the session
s.quit()

(221,
 b'2.0.0 closing connection ki19-20020a05622a771300b00401f7f23ab6sm3669866qtb.85 - gsmtp')